In [16]:
from matplotlib import pyplot as plt
import numpy as np

c = 3e8 # m/s
cavity_length = 15e-3  # m

# lambda0 = 780e-9
# lambda0_dev = 1e-9
lambda0 = 639e-9
lambda0_dev = 5e-9
# # lambda0 = 517e-9
# # lambda0_dev = 1e-9
print(f"Cental_wavelegnth: {lambda0*1e9:.2f} nm +/- {lambda0_dev*1e9:.2f} nm")

freq0 = c/lambda0
lambda_min = lambda0 - lambda0_dev
lambda_max = lambda0 + lambda0_dev
freq0_start = c/(lambda_max)
freq0_end = c/(lambda_min)

print(f"Central frequency: {freq0/1e12:.2f} THz")
print(f"Frequency range: {freq0_start/1e12:.2f} - {freq0_end/1e12:.2f} THz = {(freq0_end - freq0_start)/1e12:.2f} THz")

FSR = c/(2*cavity_length)
n_FSR_in_range = (freq0_end - freq0_start) / FSR

print(f"=== Cavity Length: {cavity_length*1e3:.2f} mm ===")
print(f"FSR: {FSR/1e9:.2f} GHz")
print(f"Number of FSR in range: {n_FSR_in_range:.2f}")


Cental_wavelegnth: 639.00 nm +/- 5.00 nm
Central frequency: 469.48 THz
Frequency range: 465.84 - 473.19 THz = 7.35 THz
=== Cavity Length: 15.00 mm ===
FSR: 10.00 GHz
Number of FSR in range: 734.76


### Diffration grating equation

\begin{equation}
d(\sin(\theta_m) + \sin(\theta_i)) = m\lambda
\end{equation}

where $d$ is diffraction grating period, $\theta_m$ is an angle of $m-th$ diffraction order, $\theta_i$ is an angle of the incident beam

Littrow configuration is often used in external cavity laser systems, which means that beam of a particular diffraction order is reflected back to the cavityy. Most often $m=1$ is used as an optical feedback due to various reasons. This means that $\theta_{m=1}$ is equal to $\theta_i$ which results in:
\begin{equation}
2d\sin(\theta_{m=1}) = m\lambda
\end{equation}

and allows to obtain equation of $\theta$ as a function of $\lambda$ and $d$:
\begin{equation}
\theta(\lambda,d)_{m=1} = \arcsin(\frac{\lambda}{2d})
\end{equation}

Which denotes diffraction grating's theoretically available angles that allow to obtain ECDL

In [17]:

def rad2deg(rad):
    return rad * 180/np.pi

def find_littrow_angle(grating_period, lambda0, m=1):
    d = grating_period
    
    # for small radians we could assume that deviation range can be calculated without lambda0
    # however, for 639 nm, arcsin's argument is 0.58, which is not small enough
    central_angle = np.arcsin((m*lambda0/(2*d)))
    return central_angle


### Angular dispersion

Angular dispersion is defined as
\begin{equation}
D = \frac{d\theta}{d\lambda}
\end{equation}

Grating equation for Littrow configuration, where incident beam angle is same as the diffracted one:
\begin{equation}
m\lambda = 2a\sin(\theta)
\end{equation}

We can now differentiate both sides of the equation with respect to $\theta$ (note that $\theta$ is a function of $\lambda$):
\begin{equation}
m = 2a\frac{d\theta}{d\lambda}\cos(\theta)
\end{equation}

\begin{equation}
\frac{d\theta}{d\lambda} = \frac{m}{2a\cos(\theta)}
\end{equation}

However, this might give false impression that the ratio $\frac{m}{a}$ maybe chosen independently of other paremeters. Let's take $\frac{m}{a}$ ratio from grating equation, because this condition must still be met:

\begin{equation}
\frac{m}{a} = \frac{2\sin(\theta)}{\lambda},
\end{equation}
so after substitution:
\begin{equation}
\frac{d\theta}{d\lambda} = \frac{\sin(\theta)}{\lambda\cos(\theta)} = \frac{1}{\lambda}\tan{\theta}
\end{equation}

In [18]:
def ang_dispersion(theta_m, lambda0):
    # returns dispersion in deg/m
    # allows to retrieve angle change as a function of wavelength
    # use 1/D_littrow to get change of wavelength as a function of angle
    D_littrow = np.tan(theta_m)/lambda0

    return D_littrow

def find_linear_range(deviation_angle, cavity_length):
    # linear range at the cavity_length distance from the grating
    return np.tan(deviation_angle) * cavity_length * 2 # 2 is because we are calculating the range from the center


def littrow_params_ecdl(grating_period, lambda0, lambda_deviation, cavity_length, n_FSR_in_range, m=1):
    central_angle = find_littrow_angle(grating_period, lambda0, m)
    deviation_angle = ang_dispersion(central_angle, lambda0) * lambda_deviation # FIXME: deviation actually means something in laser systems, not sure if used in this meaning is correct
    
    linear_range = find_linear_range(deviation_angle, cavity_length)
    linear_separation = linear_range/n_FSR_in_range # this is more or less
    
    return central_angle, deviation_angle, linear_range, linear_separation

In [19]:
gratings = [1200, 1600, 1800, 2000, 2200, 2400]  # lines/mm

# gratings = [1800]
print(f"Wavelegnth: {lambda0*1e9:.2f} nm +/- {lambda0_dev*1e9:.2f} nm")
print()
for grating in gratings:
    grating_period = 1e-3/grating
    central_angle, deviation_angle, linear_range, linear_separation = littrow_params_ecdl(grating_period, lambda0, lambda0_dev, cavity_length, n_FSR_in_range)
    disp = ang_dispersion(central_angle, lambda0)
    
    print(f"=== Grating lines/mm: {grating} period: {grating_period*1e6:.4} um ===")
    print(f"Main angle: {rad2deg(central_angle):.4f} deg +/- {rad2deg(deviation_angle):.4f} deg = {rad2deg(deviation_angle)*3600:.4f} arcsec")
    print(f"Angular dispersion [rad/nm]: {disp/1e9}; [deg/nm]: {rad2deg(disp)/1e9}")
    print(f"Linear range at `cavity_length` ({cavity_length*1e3} mm) distance from the grating: {linear_range*1e6:.4f} um")
    print(f"Linear separation: {linear_separation*1e6:.4f} um")
    print(f"Linear separation {rad2deg(deviation_angle)*3600/n_FSR_in_range:.4f} arcsec")
    print()



Wavelegnth: 639.00 nm +/- 5.00 nm

=== Grating lines/mm: 1200 period: 0.8333 um ===
Main angle: 22.5444 deg +/- 0.1861 deg = 669.9937 arcsec
Angular dispersion [rad/nm]: 0.0006496442584578266; [deg/nm]: 0.037221874194539505
Linear range at `cavity_length` (15.0 mm) distance from the grating: 97.4470 um
Linear separation: 0.1326 um
Linear separation 0.9119 arcsec

=== Grating lines/mm: 1600 period: 0.625 um ===
Main angle: 30.7438 deg +/- 0.2667 deg = 959.9720 arcsec
Angular dispersion [rad/nm]: 0.000930815091368513; [deg/nm]: 0.05333177624249989
Linear range at `cavity_length` (15.0 mm) distance from the grating: 139.6233 um
Linear separation: 0.1900 um
Linear separation 1.3065 arcsec

=== Grating lines/mm: 1800 period: 0.5556 um ===
Main angle: 35.1066 deg +/- 0.3152 deg = 1134.5933 arcsec
Angular dispersion [rad/nm]: 0.001100132711241269; [deg/nm]: 0.06303296125840921
Linear range at `cavity_length` (15.0 mm) distance from the grating: 165.0216 um
Linear separation: 0.2246 um
Linear 

In [20]:

grating = 1800 # lines/mm
print(f"Wavelegnth: {lambda0*1e9:.2f} nm +/- {lambda0_dev*1e9:.2f} nm")

central_angle, deviation_angle, linear_range, linear_separation = littrow_params_ecdl(1e-3/grating, lambda0, lambda0_dev, cavity_length, n_FSR_in_range)
print(f"Main angle: {rad2deg(central_angle)} deg +/- {rad2deg(deviation_angle)} deg = {rad2deg(deviation_angle)*3600} arcsec")
dispersion = ang_dispersion(central_angle, lambda0)
print(f"Dispersion [rad/nm]: {dispersion/1e9}, [deg/nm]: {rad2deg(dispersion)/1e9}")

deviated_angle = dispersion * lambda0_dev
print(f"Deviation angle: {rad2deg(deviated_angle)} deg")

Wavelegnth: 639.00 nm +/- 5.00 nm
Main angle: 35.106635556883944 deg +/- 0.3151648062920461 deg = 1134.5933026513658 arcsec
Dispersion [rad/nm]: 0.001100132711241269, [deg/nm]: 0.06303296125840921
Deviation angle: 0.3151648062920461 deg


### Resolution
Rayleigh criterion states that two different wavelengths $\lambda_1$ and $\lambda_2$ can be resolved if maksimum of one line is in the same spot as the first minum of the second.
$\delta\lambda$ or $(\Delta\lambda)_{min}$ is the least resolvable wavelength difference, or limit of resolution. At the limit of resolution the angular separation is half the linewidth.

Chromatic resolving power is defined as:
\begin{equation}
\mathscr{R} \equiv \frac{\lambda}{\delta\lambda}
\end{equation}

Theoretical resolving power of a planar diffraction grating is given by:
\begin{equation}
\mathscr{R} = mN
\end{equation}

One can use the grating equation to replace m:
\begin{equation}
\mathscr{R} = \frac{Na(\sin(\theta_m)+\sin(\theta_i))}{\lambda}
\end{equation}

That means that the resolving power cannot exceed $\frac{2Na}{\lambda}$ which occurs when both angles are the same ($\theta_m=\theta_in=90^{\circ}$). Greatest values of resolving power are obtained when the grating is used in an autocollimation (Littrow configuration):
\begin{equation}
\mathscr{R} = \frac{2Na\sin(\theta_i)}{\lambda}
\end{equation}


Angular width of the line:
\begin{equation}
\Delta\theta = \frac{2\lambda}{Na\cos(\theta_m)}
\end{equation}
where $N$ is number of lines covered by the beam. The angular linewidth varies inveries with the width of the grating itself $Na$.

, or:
\begin{equation}
\delta\theta = \frac{\lambda}{Na\cos{\theta_m}}
\end{equation}

\begin{equation}
\delta = \frac{\lambda^2}{Na(\sin(\theta_i) + \sin(\theta_m))}
\end{equation}



\begin{equation}
\mathscr{R} = \frac{Na(\sin(\theta_i)+\sin(\theta_m))}{\lambda}
\end{equation}
which :



In [22]:
N = 1000
ang_width = 2*lambda0/(N*grating_period*np.cos(central_angle))
print(ang_width, rad2deg(ang_width))

0.003749252279910245 0.2148163319686586


### Wavelength
Emitted wavelength is given with 
\begin{equation}
\lambda = \frac{2nl}{m}
\end{equation}

where $m$ is the longitudonal mode number, $l$ is laser's cavity length and $n$ is refractive index of the diode material (oftentimes $n=0.3$)

